In [ ]:
from corrective_rag import corrective_rag_translated
from simple_agent_rag import simple_agent_rag, simple_agent_rag_translated
from naive_rag import naive_rag_translated, naive_rag
from react_rag import react_rag, react_rag_translated
from init_vectorstore import init_vectorstore
from evaluate_model import evaluate_model
from ensemble_model import ensemble_models
from semantic_model import semantic_model
from init_vectorstore import init_semantic_vectorstore
import os
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from datasets import load_dataset
import matplotlib.pyplot as plt

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-NyOIGT3EsORVYyYth6kUT3BlbkFJG4Hf0tNzfhB4BFZMgMXT"

In [ ]:
instances = 10
file_path = "/Users/adrianfolge/Documents/lokal:skole/Master/data/synthetic_data/vol2_questions_and_answers_ytterligere_revidert.csv"

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
databases = init_vectorstore(embeddings, text_splitter)
databases_translated = init_vectorstore(embeddings, text_splitter, translate=True)
semantic_databases = init_semantic_vectorstore()

In [ ]:
corrective_rag_translated_list = corrective_rag_translated(instances, file_path, databases_translated)
simple_agent_rag_list = simple_agent_rag(instances, file_path, databases)
simple_agent_rag_translated_list = simple_agent_rag_translated(instances, file_path, databases_translated)
naive_rag_list = naive_rag(instances, file_path, databases)
naive_rag_translated_list = naive_rag_translated(instances, file_path, databases_translated)
react_rag_translated_list = react_rag_translated(instances, file_path, databases_translated)
react_rag_list, list_of_context = react_rag(instances, file_path, databases)
semantic_preds = semantic_model(instances, file_path, semantic_databases)

In [ ]:
references = load_dataset('csv', data_files=file_path, split=f"train[:{instances}]")
refs = references["svar"]
print("Scores for corrective RAG")
corrective_rag_score = evaluate_model(corrective_rag_translated_list, refs, instances)
print("#############")
print("Scores for simple agent")
simple_agent_score = evaluate_model(simple_agent_rag_list, refs, instances)
print("#############")
print("Scores for simple agent translated")
simple_agent_translated_score = evaluate_model(simple_agent_rag_translated_list, refs, instances)
print("#############")
print("Scores for naive rag")
naive_rag_score = evaluate_model(naive_rag_list, refs, instances)
print("#############")
print("Scores for naive rag translated")
naive_rag_translated_score = evaluate_model(naive_rag_translated_list, refs, instances)
print("#############")
print("Scores for react RAG")
react_rag_score = evaluate_model(react_rag_list, refs, instances)
print("#############")
print("Scores for react RAG translated")
react_rag_translated_score = evaluate_model(react_rag_translated_list, refs, instances)
print("#############")

In [ ]:
def create_predictions_dict(*lists):
    predictions_dict = {}
    for i, lst in enumerate(lists, start=1):
        key = f"Prediction {i}"
        predictions_dict[key] = lst
    return predictions_dict

In [ ]:
predictions_dict = create_predictions_dict(corrective_rag_translated_list, simple_agent_rag_list, simple_agent_rag_translated_list, naive_rag_list, naive_rag_translated_list, react_rag_translated_list, react_rag_list)
ensembling_models = ensemble_models(predictions_dict, references, instances, list_of_context)
print("Scores for the ensembled model")
ensemble_models_score = evaluate_model(ensembling_models, refs, instances)
print("#############")

In [ ]:
# Scores for each model (example data)
model_names = ["Corrective RAG", "Simple agent", "Simple agent translated", "Naive rag", "Naive rag translated", "React rag", "React rag translated", "Model ensembling"]
scores = [corrective_rag_score, simple_agent_score, simple_agent_translated_score, naive_rag_score, naive_rag_translated_score, react_rag_score, react_rag_translated_score, ensemble_models_score]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(model_names, scores, color='skyblue')
plt.xlabel('Models')
plt.ylabel('Scores')
plt.title('Scores of Different Models')
plt.ylim(0, 1)  # Set y-axis limits to 0 and 1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.show()